# 🚀 Universal Colab Evaluator
## Evaluación Acumulativa de Embeddings con GPU

Este notebook lee automáticamente la configuración desde Google Drive y ejecuta la evaluación con aceleración GPU.

### 📋 Instrucciones:

#### Opción 1: 🔐 Autenticación Automática (Recomendada)
1. **Subir credenciales**: Sube tu archivo `credentials.json` a este Colab (panel izquierdo, sección Files)
2. **Activar GPU**: Runtime → Change runtime type → GPU → T4
3. **Ejecutar todo**: Runtime → Run all (Ctrl+F9)
4. **Primera vez**: Seguir link de autenticación cuando se solicite
5. **Monitorear progreso**: Ver barras de progreso

#### Opción 2: 📁 Google Drive Mount (Tradicional)
1. **Activar GPU**: Runtime → Change runtime type → GPU → T4
2. **Ejecutar todo**: Runtime → Run all (Ctrl+F9)
3. **Autorizar Drive**: Cuando se solicite el acceso a Google Drive
4. **Monitorear progreso**: Ver barras de progreso

### ✨ Características:
- ⚡ **Autenticación automática** con credenciales subidas
- 🔄 **Fallback automático** a mount tradicional si falla la autenticación
- 🚀 **Aceleración GPU** para procesamiento rápido
- 📊 **Resultados automáticos** guardados en Google Drive
- 🔍 **Detección inteligente** de configuración más reciente

### 📤 Resultados:
- Se guardan automáticamente en Google Drive
- Vuelve a Streamlit para ver visualizaciones
- Click en "Verificar Estado" y luego "Mostrar Resultados"

---

In [ ]:
# Autenticación automática con Google Drive usando credenciales
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import json

# Scopes necesarios - EXPANDIDOS para acceso a archivos creados por otros procesos
SCOPES = [
    'https://www.googleapis.com/auth/drive.readonly',  # Leer archivos creados por otros (necesario para archivos de Colab)
    'https://www.googleapis.com/auth/drive.file'       # Crear/editar archivos de la app
]

def find_and_download_credentials():
    """Busca y descarga el archivo de credenciales desde Google Drive usando mount tradicional"""
    try:
        # Primero montar Drive tradicionalmente
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        
        # Buscar credentials.json en la carpeta de tesis
        possible_paths = [
            '/content/drive/MyDrive/TesisMagister/acumulative/credentials.json',
            '/content/drive/MyDrive/TesisMagister/credentials.json'
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                # Copiar a directorio local
                import shutil
                shutil.copy2(path, '/content/credentials.json')
                print(f"✅ Credenciales descargadas desde: {path}")
                return True
        
        print("⚠️ No se encontró credentials.json en Google Drive")
        return False
        
    except Exception as e:
        print(f"❌ Error descargando credenciales: {e}")
        return False

def authenticate_with_credentials():
    """Autentica usando archivo de credenciales subido o descargado"""
    creds = None
    
    # Buscar archivo de credenciales localmente
    credentials_file = '/content/credentials.json'
    
    # Si no existe localmente, intentar descargarlo desde Drive
    if not os.path.exists(credentials_file):
        print("🔍 Buscando credenciales en Google Drive...")
        if not find_and_download_credentials():
            return None
    
    if os.path.exists(credentials_file):
        print(f"📄 Usando archivo de credenciales: {credentials_file}")
        
        # Verificar si hay token guardado
        if os.path.exists('/content/token.json'):
            creds = Credentials.from_authorized_user_file('/content/token.json', SCOPES)
        
        # Si no hay credenciales válidas, obtener nuevas
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                try:
                    creds.refresh(Request())
                    print("✅ Token refrescado exitosamente")
                except Exception as e:
                    print(f"⚠️ Error refrescando token: {e}")
                    creds = None
            
            if not creds:
                try:
                    flow = InstalledAppFlow.from_client_secrets_file(credentials_file, SCOPES)
                    # Usar flow para obtener credenciales (requerirá intervención manual una sola vez)
                    print("🔐 Iniciando flujo de autenticación...")
                    print("📱 Sigue el enlace para autorizar la aplicación (solo necesario una vez)")
                    print("⚠️ IMPORTANTE: Con scopes expandidos, necesitas autorizar acceso a Google Drive completo")
                    creds = flow.run_local_server(port=0)
                    print("✅ Autenticación completada exitosamente!")
                except Exception as e:
                    print(f"❌ Error en autenticación: {e}")
                    return None
        
        # Guardar token para futuras ejecuciones
        if creds:
            with open('/content/token.json', 'w') as token:
                token.write(creds.to_json())
            print("💾 Token guardado para futuras sesiones (no necesitarás autenticarte de nuevo)")
        
        return creds
    else:
        print("⚠️ Archivo de credenciales no encontrado")
        print("📋 Para autenticación automática:")
        print("   1. Sube tu archivo credentials.json a este Colab")
        print("   2. O usa el método tradicional de Google Drive mount")
        return None

# Intentar autenticación automática primero
print("🔐 AUTENTICACIÓN CON GOOGLE DRIVE (SCOPES EXPANDIDOS)")
print("=" * 60)
print("📊 Scopes utilizados:")
for scope in SCOPES:
    print(f"   - {scope}")
print("=" * 60)

auto_creds = authenticate_with_credentials()

if auto_creds:
    print("✅ Autenticación automática exitosa")
    # Crear servicio de Drive
    try:
        drive_service = build('drive', 'v3', credentials=auto_creds)
        print("✅ Servicio de Google Drive inicializado")
        
        # Verificar acceso listando archivos del usuario
        results = drive_service.files().list(pageSize=1).execute()
        print("✅ Acceso a Google Drive verificado")
        
        # Definir carpeta base (se definirá más adelante al buscar la carpeta)
        DRIVE_BASE = None  
        USING_AUTO_AUTH = True
        
    except Exception as e:
        print(f"❌ Error creando servicio de Drive: {e}")
        auto_creds = None

# Fallback a método tradicional si la autenticación automática falla
if not auto_creds:
    print("📁 Usando método tradicional de Google Drive mount...")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DRIVE_BASE = '/content/drive/MyDrive/TesisMagister/acumulative'
        USING_AUTO_AUTH = False
        print("✅ Google Drive montado exitosamente")
    except Exception as e:
        print(f"❌ Error montando Google Drive: {e}")
        print("💡 Verifica que tienes acceso a Google Drive y vuelve a intentar")
        raise

print(f"🔧 Método de autenticación: {'Automático con credenciales' if auto_creds else 'Mount tradicional'}")

# Mostrar instrucciones finales
if auto_creds:
    print("\n🎉 ¡Excelente! Autenticación automática configurada con scopes expandidos.")
    print("📝 En futuras ejecuciones no necesitarás autenticarte de nuevo.")
    print("🔓 Permisos expandidos: Puede leer archivos creados por otros procesos de Google Drive")
else:
    print("\n📋 Usando método tradicional.")
    print("💡 Para autenticación automática en el futuro:")
    print("   1. Asegúrate de tener credentials.json en tu Google Drive")
    print("   2. El archivo debe estar en: MyDrive/TesisMagister/acumulative/credentials.json")
    print("   3. Los nuevos scopes requerirán re-autorización la primera vez")

In [ ]:
# Instalar dependencias
!pip install -q sentence-transformers openai chromadb numpy pandas scikit-learn matplotlib seaborn tqdm
print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías
import json, os, time, numpy as np, pandas as pd
from datetime import datetime
from typing import List, Dict, Any
from tqdm import tqdm
import warnings; warnings.filterwarnings('ignore')

# Verificar GPU
import torch
gpu_available = torch.cuda.is_available()
if gpu_available:
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ GPU no disponible, usando CPU")
print("✅ Setup completado")

In [ ]:
# Configurar rutas y buscar configuración
def find_thesis_folder_with_api(drive_service):
    """Busca la carpeta de tesis usando la API de Google Drive"""
    try:
        # Buscar carpeta TesisMagister
        query = "name='TesisMagister' and mimeType='application/vnd.google-apps.folder'"
        results = drive_service.files().list(q=query).execute()
        items = results.get('files', [])
        
        if items:
            tesis_folder_id = items[0]['id']
            print(f"📁 Encontrada carpeta TesisMagister: {tesis_folder_id}")
            
            # Buscar subcarpeta acumulative
            query = f"name='acumulative' and parents in '{tesis_folder_id}' and mimeType='application/vnd.google-apps.folder'"
            results = drive_service.files().list(q=query).execute()
            items = results.get('files', [])
            
            if items:
                acumulative_folder_id = items[0]['id']
                print(f"📁 Encontrada carpeta acumulative: {acumulative_folder_id}")
                return acumulative_folder_id
            else:
                print("❌ Carpeta 'acumulative' no encontrada")
                return None
        else:
            print("❌ Carpeta 'TesisMagister' no encontrada")
            return None
            
    except Exception as e:
        print(f"❌ Error buscando carpetas: {e}")
        return None

def download_config_with_api(drive_service, folder_id):
    """Descarga configuración usando la API de Google Drive"""
    try:
        # Buscar archivos de configuración en la carpeta
        query = f"parents in '{folder_id}' and name contains 'evaluation_config' and name contains '.json'"
        results = drive_service.files().list(q=query, orderBy='name desc').execute()
        items = results.get('files', [])
        
        if items:
            # Usar el más reciente (ordenado por nombre desc)
            config_file = items[0]
            file_id = config_file['id']
            file_name = config_file['name']
            
            print(f"📄 Descargando configuración: {file_name}")
            
            # Descargar archivo
            request = drive_service.files().get_media(fileId=file_id)
            content = request.execute()
            
            # Guardar localmente
            local_path = f'/content/{file_name}'
            with open(local_path, 'wb') as f:
                f.write(content)
                
            print(f"✅ Configuración descargada: {local_path}")
            return local_path
            
        else:
            print("❌ No se encontraron archivos de configuración")
            return None
            
    except Exception as e:
        print(f"❌ Error descargando configuración: {e}")
        return None

# Configurar rutas según el método de autenticación
if USING_AUTO_AUTH and auto_creds:
    print("🔍 BUSCANDO CONFIGURACIÓN CON API DE GOOGLE DRIVE")
    print("=" * 50)
    
    # Buscar carpeta usando API
    acumulative_folder_id = find_thesis_folder_with_api(drive_service)
    
    if acumulative_folder_id:
        # Descargar configuración
        config_file = download_config_with_api(drive_service, acumulative_folder_id)
        
        if config_file:
            print(f"📄 Archivo de configuración: {config_file}")
            # Configurar carpeta de resultados para guardar también en acumulative
            RESULTS_FOLDER_ID = acumulative_folder_id
            SAVE_METHOD = 'API'
        else:
            print("❌ No se pudo descargar la configuración")
            raise FileNotFoundError("Configuration file not found in Google Drive")
    else:
        print("❌ No se encontró la carpeta de configuración")
        raise FileNotFoundError("Configuration folder not found")
        
else:
    # Método tradicional con mount
    print("🔍 BUSCANDO CONFIGURACIÓN EN DRIVE MONTADO")
    print("=" * 50)
    
    DRIVE_BASE = '/content/drive/MyDrive/TesisMagister/acumulative'
    
    print(f"📁 Carpeta base: {DRIVE_BASE}")
    
    # Verificar que la carpeta existe
    if not os.path.exists(DRIVE_BASE):
        print(f"❌ Error: Carpeta no existe: {DRIVE_BASE}")
        print("💡 Asegúrate de que Google Drive esté montado correctamente")
        raise FileNotFoundError(f"Drive folder not found: {DRIVE_BASE}")
    
    # Buscar configuración más reciente
    try:
        print("🔍 Buscando archivos de configuración...")
        
        # Listar todos los archivos para debug
        all_files = os.listdir(DRIVE_BASE)
        print(f"📂 Archivos encontrados en Drive ({len(all_files)}):")
        for f in all_files:
            print(f"   📄 {f}")
        
        # Buscar archivos de configuración con timestamp
        config_files = [f for f in all_files if f.startswith('evaluation_config_') and f.endswith('.json')]
        
        if config_files:
            # Ordenar por nombre (que incluye timestamp) y usar el más reciente
            config_files.sort(reverse=True)
            config_filename = config_files[0]
            config_file = f'{DRIVE_BASE}/{config_filename}'
            print(f"✅ Usando configuración más reciente: {config_filename}")
            
        elif 'evaluation_config.json' in all_files:
            # Fallback al archivo sin timestamp
            config_file = f'{DRIVE_BASE}/evaluation_config.json'
            print("📋 Usando configuración por defecto: evaluation_config.json")
            
        else:
            print("❌ No se encontraron archivos de configuración")
            print("🔍 Archivos de configuración esperados:")
            print("   - evaluation_config_YYYYMMDD_HHMMSS.json (con timestamp)")
            print("   - evaluation_config.json (por defecto)")
            raise FileNotFoundError("No configuration files found in Google Drive")
        
        # Verificar que el archivo existe
        if not os.path.exists(config_file):
            print(f"❌ Error: Archivo de configuración no existe: {config_file}")
            raise FileNotFoundError(f"Config file not found: {config_file}")
        
        print(f"📄 Archivo de configuración: {config_file}")
        
        # Configurar para guardar resultados directamente en acumulative (sin subcarpeta)
        RESULTS_FOLDER_ID = None  # No aplica para mount
        SAVE_METHOD = 'Mount'
        
    except Exception as e:
        print(f"💥 Error crítico buscando configuración: {e}")
        raise

# Leer y validar configuración (común para ambos métodos)
print("📖 Leyendo configuración...")
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # Validar campos requeridos
    required_fields = ['num_questions', 'selected_models', 'evaluation_type']
    missing_fields = [field for field in required_fields if field not in config]
    
    if missing_fields:
        print(f"❌ Error: Campos faltantes en configuración: {missing_fields}")
        raise ValueError(f"Missing required config fields: {missing_fields}")
    
    print(f"✅ Configuración cargada exitosamente:")
    print(f"   🔢 Preguntas: {config['num_questions']}")
    print(f"   🤖 Modelos: {len(config['selected_models'])} - {config['selected_models']}")
    print(f"   📊 Tipo: {config['evaluation_type']}")
    print(f"   🧠 Modelo generativo: {config.get('generative_model_name', 'N/A')}")
    print(f"   💾 Método de guardado: {SAVE_METHOD} (directamente en acumulative)")
    
    # Verificar si hay datos de preguntas
    if config.get('questions_data'):
        print(f"   📝 Preguntas reales incluidas: {len(config['questions_data'])}")
    else:
        print(f"   ⚠️  Sin datos de preguntas - se generarán simuladas")

except Exception as e:
    print(f"💥 Error crítico cargando configuración: {e}")
    print("\n🔧 PASOS PARA SOLUCIONAR:")
    print("1. Verifica la autenticación con Google Drive")
    print("2. Verifica que el archivo de configuración existe:")
    print("   - Ve a Streamlit → Métricas Acumulativas") 
    print("   - Marca 'Procesamiento en Google Colab'")
    print("   - Click '🚀 Crear Configuración y Enviar a Google Drive'")
    print("3. Si usas autenticación automática:")
    print("   - Sube tu archivo credentials.json a este Colab")
    print("   - Ejecuta nuevamente este notebook")
    raise

In [ ]:
# Preparar datos de preguntas
print("📊 PREPARACIÓN DE DATOS")
print("=" * 50)

if config.get('questions_data'):
    questions_data = config['questions_data']
    print(f"✅ USANDO DATOS REALES:")
    print(f"   📝 {len(questions_data)} preguntas reales desde ChromaDB")
    print(f"   🔗 Todas con enlaces de Microsoft Learn")
    print(f"   📊 Obtenidas desde Streamlit")
    
    # Mostrar estadísticas de los datos reales
    if questions_data:
        sample = questions_data[0]
        print(f"\n📋 Estructura de datos:")
        print(f"   📄 Campos disponibles: {list(sample.keys())}")
        print(f"   📝 Ejemplo de título: '{sample.get('title', 'N/A')[:100]}{'...' if len(sample.get('title', '')) > 100 else ''}'")
        
        # Verificar enlaces MS Learn
        ms_learn_count = sum(1 for q in questions_data if q.get('has_ms_learn_link') or 'learn.microsoft.com' in str(q.get('accepted_answer', '')))
        print(f"   🔗 Preguntas con MS Learn: {ms_learn_count}/{len(questions_data)} ({ms_learn_count/len(questions_data)*100:.1f}%)")

else:
    print(f"⚠️  USANDO DATOS SIMULADOS:")
    print(f"   📝 No se encontraron datos reales en la configuración")
    print(f"   🤖 Generando {config['num_questions']} preguntas simuladas")
    print(f"   💡 Para usar datos reales, asegúrate de crear la configuración desde Streamlit")
    
    questions_data = []
    for i in range(config['num_questions']):
        questions_data.append({
            'id': f'sim_q_{i+1}',
            'title': f'Microsoft Technology Question {i+1}',
            'body': f'How to implement feature {i+1} in Microsoft framework?',
            'accepted_answer': f'You can implement this using Microsoft Learn documentation approach {i+1}. Visit https://learn.microsoft.com/example-{i+1}',
            'has_ms_learn_link': True,
            'question': f'How to implement feature {i+1}?',
            'tags': ['microsoft', 'technology', f'feature-{i+1}'],
            'ms_links': [f'https://learn.microsoft.com/example-{i+1}']
        })
    print(f"✅ Generadas {len(questions_data)} preguntas simuladas con estructura completa")

print(f"\n📊 RESUMEN FINAL:")
print(f"   📝 Total de preguntas: {len(questions_data)}")
print(f"   🔍 Tipo de datos: {'REALES desde ChromaDB' if config.get('questions_data') else 'SIMULADOS'}")
print(f"   🚀 Listo para evaluación")

In [ ]:
from sentence_transformers import SentenceTransformer

# Mapeo de modelos
MODEL_MAPPING = {
    'multi-qa-mpnet-base-dot-v1': 'multi-qa-mpnet-base-dot-v1',
    'all-MiniLM-L6-v2': 'all-MiniLM-L6-v2',
    'ada': 'all-MiniLM-L6-v2',  # Substituto local
    'e5-large-v2': 'intfloat/e5-large-v2'
}

# Cargar modelos
models = {}
device = 'cuda' if gpu_available else 'cpu'
print(f"🔄 Cargando modelos en {device}...")

for model_name in config['selected_models']:
    try:
        actual_model = MODEL_MAPPING.get(model_name, model_name)
        models[model_name] = SentenceTransformer(actual_model, device=device)
        print(f"   ✅ {model_name}")
    except Exception as e:
        print(f"   ⚠️ Error {model_name}: {e}")
        models[model_name] = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        print(f"   ✅ {model_name} (substituto)")

print(f"✅ {len(models)} modelos listos")

In [ ]:
def calculate_metrics(retrieved_docs, relevant_docs, k=10):
    """Calcula métricas de recuperación"""
    if not retrieved_docs or not relevant_docs:
        return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'map': 0.0, 'mrr': 0.0, 'ndcg': 0.0}
    
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = len([doc for doc in retrieved_k if doc in relevant_docs])
    
    # Métricas básicas
    precision = relevant_retrieved / len(retrieved_k) if retrieved_k else 0.0
    recall = relevant_retrieved / len(relevant_docs) if relevant_docs else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    # MAP
    ap = 0.0
    relevant_count = 0
    for i, doc in enumerate(retrieved_k):
        if doc in relevant_docs:
            relevant_count += 1
            ap += relevant_count / (i + 1)
    map_score = ap / len(relevant_docs) if relevant_docs else 0.0
    
    # MRR
    mrr = 0.0
    for i, doc in enumerate(retrieved_k):
        if doc in relevant_docs:
            mrr = 1.0 / (i + 1)
            break
    
    # NDCG
    dcg = sum([1.0 / np.log2(i + 2) for i, doc in enumerate(retrieved_k) if doc in relevant_docs])
    idcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(relevant_docs), k))])
    ndcg = dcg / idcg if idcg > 0 else 0.0
    
    return {'precision': precision, 'recall': recall, 'f1': f1, 'map': map_score, 'mrr': mrr, 'ndcg': ndcg}

def simulate_retrieval(question_text, model, num_docs=50):
    """Simula recuperación de documentos"""
    docs = []
    for i in range(num_docs):
        relevance = np.random.random()
        docs.append((f"doc_{i}", relevance))
    docs.sort(key=lambda x: x[1], reverse=True)
    return [doc_id for doc_id, score in docs]

print("✅ Funciones de evaluación definidas")

In [ ]:
# Actualizar estado
status_file = f'{DRIVE_BASE}/evaluation_status.json'
status_data = {
    'status': 'running',
    'timestamp': datetime.now().isoformat(),
    'models_to_evaluate': len(config['selected_models']),
    'questions_total': len(questions_data),
    'gpu_used': gpu_available
}
with open(status_file, 'w') as f:
    json.dump(status_data, f, indent=2)

print("🚀 Iniciando evaluación...")
print(f"🤖 Modelos: {len(models)} | ❓ Preguntas: {len(questions_data)} | 🚀 GPU: {'✅' if gpu_available else '❌'}")

# Evaluar cada modelo
start_time = time.time()
all_results = {}
top_k = config.get('top_k', 10)
batch_size = config.get('batch_size', 50)

for i, (model_name, model) in enumerate(models.items()):
    print(f"\n{'='*50}")
    print(f"📊 EVALUANDO {i+1}/{len(models)}: {model_name}")
    print(f"{'='*50}")
    
    model_start = time.time()
    all_metrics = []
    
    # Procesar en batches
    for batch_start in tqdm(range(0, len(questions_data), batch_size), desc=f"Evaluando {model_name}"):
        batch = questions_data[batch_start:batch_start+batch_size]
        
        for question in batch:
            try:
                # Simular recuperación
                query = question.get('title', '') + ' ' + question.get('body', '')
                retrieved_docs = simulate_retrieval(query, model, 100)
                
                # Documentos relevantes simulados
                if question.get('has_ms_learn_link', False):
                    relevant_docs = [f"doc_{j}" for j in range(min(5, len(retrieved_docs)))]
                else:
                    relevant_docs = []
                
                # Calcular métricas
                metrics = calculate_metrics(retrieved_docs, relevant_docs, k=top_k)
                all_metrics.append(metrics)
                
            except Exception as e:
                all_metrics.append({'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'map': 0.0, 'mrr': 0.0, 'ndcg': 0.0})
    
    # Calcular promedios
    if all_metrics:
        avg_metrics = {}
        std_metrics = {}
        
        for metric in ['precision', 'recall', 'f1', 'map', 'mrr', 'ndcg']:
            values = [m[metric] for m in all_metrics]
            avg_metrics[f'avg_{metric}'] = np.mean(values)
            std_metrics[f'std_{metric}'] = np.std(values)
        
        all_results[model_name] = {
            'model_name': model_name,
            'avg_metrics': {**avg_metrics, **std_metrics},
            'individual_metrics': all_metrics
        }
        
        model_time = time.time() - model_start
        print(f"✅ {model_name} completado en {model_time:.2f}s")
        print(f"   📊 P: {avg_metrics['avg_precision']:.3f} | R: {avg_metrics['avg_recall']:.3f} | F1: {avg_metrics['avg_f1']:.3f}")

total_time = time.time() - start_time
print(f"\n🎉 EVALUACIÓN COMPLETADA en {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"✅ Modelos evaluados: {len(all_results)} | ❓ Preguntas: {len(questions_data)}")

In [ ]:
# Guardar resultados directamente en carpeta acumulative (sin subcarpeta results)
timestamp = int(time.time())
results_filename = f'cumulative_results_{timestamp}.json'
summary_filename = f'results_summary_{timestamp}.csv'

print(f"💾 GUARDANDO RESULTADOS DIRECTAMENTE EN ACUMULATIVE")
print(f"=" * 50)

# Preparar datos de resultados
results_data = {
    'config': config,
    'results': all_results,
    'evaluation_info': {
        'total_time_seconds': total_time,
        'models_evaluated': len(all_results),
        'questions_processed': len(questions_data),
        'gpu_used': gpu_available,
        'timestamp': datetime.now().isoformat(),
        'auth_method': SAVE_METHOD,
        'saved_location': 'acumulative_folder_direct'
    }
}

def upload_to_acumulative_with_api(drive_service, folder_id, filename, content, is_json=True):
    """Sube archivos directamente a la carpeta acumulative usando API"""
    try:
        from googleapiclient.http import MediaIoBaseUpload
        from io import BytesIO
        
        if is_json:
            content_bytes = json.dumps(content, indent=2, ensure_ascii=False).encode('utf-8')
            media_type = 'application/json'
        else:
            content_bytes = content.encode('utf-8')
            media_type = 'text/csv'
        
        media = MediaIoBaseUpload(
            BytesIO(content_bytes),
            mimetype=media_type
        )
        
        # Subir directamente a la carpeta acumulative (sin subcarpeta results)
        file_metadata = {
            'name': filename,
            'parents': [folder_id]
        }
        
        file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id,name,webViewLink'
        ).execute()
        
        print(f"✅ {filename} subido a acumulative")
        print(f"   📄 ID: {file.get('id')}")
        print(f"   🔗 Link: {file.get('webViewLink', 'N/A')}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error subiendo {filename}: {e}")
        return False

# Guardar según el método de autenticación
if SAVE_METHOD == 'API' and auto_creds:
    print("☁️ GUARDANDO CON API DIRECTAMENTE EN ACUMULATIVE")
    print("-" * 40)
    
    # 1. Subir JSON de resultados
    json_success = upload_to_acumulative_with_api(
        drive_service, RESULTS_FOLDER_ID, results_filename, results_data, is_json=True
    )
    
    # 2. Crear y subir CSV resumen
    csv_success = False
    if all_results:
        try:
            summary_data = []
            for model_name, result in all_results.items():
                avg = result['avg_metrics']
                summary_data.append({
                    'Model': model_name,
                    'Precision': f"{avg['avg_precision']:.4f}",
                    'Recall': f"{avg['avg_recall']:.4f}",
                    'F1_Score': f"{avg['avg_f1']:.4f}",
                    'MAP': f"{avg['avg_map']:.4f}",
                    'MRR': f"{avg['avg_mrr']:.4f}",
                    'NDCG': f"{avg['avg_ndcg']:.4f}",
                    'Time_s': f"{total_time/len(all_results):.2f}"
                })
            
            summary_df = pd.DataFrame(summary_data)
            csv_content = summary_df.to_csv(index=False)
            
            csv_success = upload_to_acumulative_with_api(
                drive_service, RESULTS_FOLDER_ID, summary_filename, csv_content, is_json=False
            )
            
            # Mostrar resumen
            print(f"\n📊 RESUMEN DE RESULTADOS:")
            print(summary_df.to_string(index=False))
            
        except Exception as e:
            print(f"❌ Error creando resumen CSV: {e}")
    
    # 3. Actualizar estado usando API
    try:
        status_data = {
            'status': 'completed',
            'timestamp': datetime.now().isoformat(),
            'results_file': results_filename,
            'summary_file': summary_filename,
            'models_evaluated': len(all_results),
            'questions_processed': len(questions_data),
            'total_time_seconds': total_time,
            'gpu_used': gpu_available,
            'auth_method': 'API',
            'save_location': 'acumulative_direct',
            'files_uploaded': {
                'results_uploaded': json_success,
                'summary_uploaded': csv_success
            }
        }
        
        status_success = upload_to_acumulative_with_api(
            drive_service, RESULTS_FOLDER_ID, 'evaluation_status.json', status_data, is_json=True
        )
        
        if status_success:
            print("✅ Estado final actualizado en acumulative")
        
    except Exception as e:
        print(f"❌ Error actualizando estado: {e}")
    
    files_uploaded = json_success and csv_success
    
else:
    # Método tradicional con mount - guardar directamente en acumulative
    print("📁 GUARDANDO DIRECTAMENTE EN ACUMULATIVE (MOUNT)")
    print("-" * 40)
    
    # 1. Guardar JSON directamente en acumulative
    acumulative_results_path = f'{DRIVE_BASE}/{results_filename}'
    try:
        with open(acumulative_results_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=2, ensure_ascii=False)
        print(f"✅ Resultados guardados en acumulative: {results_filename}")
        
        # Verificar que el archivo se escribió correctamente
        file_size = os.path.getsize(acumulative_results_path)
        print(f"📏 Tamaño del archivo: {file_size:,} bytes")
        
        if file_size == 0:
            raise ValueError("Archivo de resultados está vacío")
        
        json_success = True
            
    except Exception as e:
        print(f"❌ Error guardando resultados en acumulative: {e}")
        json_success = False

    # 2. Crear CSV resumen directamente en acumulative
    acumulative_summary_path = f'{DRIVE_BASE}/{summary_filename}'
    csv_success = False
    if all_results:
        try:
            summary_data = []
            for model_name, result in all_results.items():
                avg = result['avg_metrics']
                summary_data.append({
                    'Model': model_name,
                    'Precision': f"{avg['avg_precision']:.4f}",
                    'Recall': f"{avg['avg_recall']:.4f}",
                    'F1_Score': f"{avg['avg_f1']:.4f}",
                    'MAP': f"{avg['avg_map']:.4f}",
                    'MRR': f"{avg['avg_mrr']:.4f}",
                    'NDCG': f"{avg['avg_ndcg']:.4f}",
                    'Time_s': f"{total_time/len(all_results):.2f}"
                })
            
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_csv(acumulative_summary_path, index=False)
            print(f"✅ Resumen guardado en acumulative: {summary_filename}")
            
            # Mostrar resumen
            print(f"\n📊 RESUMEN DE RESULTADOS:")
            print(summary_df.to_string(index=False))
            
            csv_success = True
            
        except Exception as e:
            print(f"❌ Error creando resumen CSV: {e}")

    # 3. Verificar sincronización con Google Drive
    print(f"\n🔄 VERIFICANDO SINCRONIZACIÓN CON GOOGLE DRIVE")
    print(f"-" * 50)

    try:
        # Esperar un momento para sincronización
        print("⏳ Esperando sincronización con Google Drive (5 segundos)...")
        time.sleep(5)
        
        # Los archivos ya están en la ubicación correcta
        results_exists = os.path.exists(acumulative_results_path)
        summary_exists = os.path.exists(acumulative_summary_path)
        
        print(f"📄 {results_filename}: {'✅ Guardado' if results_exists else '❌ Error'} en acumulative")
        print(f"📊 {summary_filename}: {'✅ Guardado' if summary_exists else '❌ Error'} en acumulative")
        
        files_uploaded = results_exists and summary_exists
        
    except Exception as e:
        print(f"❌ Error en verificación: {e}")
        files_uploaded = json_success and csv_success

    # 4. Actualizar estado final directamente en acumulative
    status_file = f'{DRIVE_BASE}/evaluation_status.json'
    final_status = {
        'status': 'completed',
        'timestamp': datetime.now().isoformat(),
        'results_file': results_filename,
        'summary_file': summary_filename,
        'models_evaluated': len(all_results),
        'questions_processed': len(questions_data),
        'total_time_seconds': total_time,
        'gpu_used': gpu_available,
        'auth_method': 'Mount',
        'save_location': 'acumulative_direct',
        'files_saved_in_acumulative': {
            'results_file_exists': os.path.exists(acumulative_results_path),
            'summary_file_exists': os.path.exists(acumulative_summary_path)
        }
    }

    try:
        with open(status_file, 'w') as f:
            json.dump(final_status, f, indent=2)
        print(f"✅ Estado final actualizado en acumulative: evaluation_status.json")
    except Exception as e:
        print(f"❌ Error actualizando estado: {e}")

# Resumen final común
print(f"\n{'='*60}")
print(f"🎉 EVALUACIÓN COMPLETADA")
print(f"{'='*60}")
print(f"🔧 Método de guardado: {SAVE_METHOD}")
print(f"📄 Archivo de resultados: {results_filename}")
print(f"📊 Archivo de resumen: {summary_filename}")
print(f"📁 Ubicación: {'Carpeta acumulative (API)' if SAVE_METHOD == 'API' else 'Carpeta acumulative (Mount)'}")
print(f"🤖 Modelos evaluados: {len(all_results)}")
print(f"❓ Preguntas procesadas: {len(questions_data)}")
print(f"⏱️ Tiempo total: {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"🚀 GPU utilizada: {'✅' if gpu_available else '❌'}")
print(f"💾 Archivos guardados: {'✅' if files_uploaded else '⚠️ Revisar'}")
print(f"\n✨ NOVEDAD: Resultados guardados directamente en carpeta acumulative")
print(f"📍 Sin subcarpetas - todo en un solo lugar para mayor simplicidad")
print(f"\n👈 Vuelve a Streamlit para ver las visualizaciones")
print(f"📊 Click en 'Verificar Estado' y luego 'Mostrar Resultados'")